In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess dataset
file_path = "/content/creditcard.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# Handle missing values in the target column
data = data.dropna(subset=['Class'])  # Drop rows with NaN in 'Class'
# Alternative: Fill missing values with 0
# data['Class'] = data['Class'].fillna(0)

# Separate features and labels
X = data.drop(columns=['Class'])
y = data['Class']

# Normalize features (excluding Time)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.drop(columns=['Time']))
X_scaled = pd.DataFrame(X_scaled, columns=X.columns[1:])
X_scaled['Time'] = X['Time']  # Add Time back unscaled

# Verify no NaN values in target
print(y.isnull().sum())  # Ensure there are no NaN values

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Create federated clients
num_clients = 3
client_data = []
split_X = np.array_split(X_train, num_clients)
split_y = np.array_split(y_train, num_clients)
for i in range(num_clients):
    client_data.append({'X': split_X[i].values, 'y': split_y[i].values})

# Define test data
test_data = {'X': X_test.values, 'y': y_test.values}

# Define model
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(input_dim,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

def train_fedavg(global_model, client_datasets, test_data, num_rounds, local_epochs, learning_rate):
    global_weights = global_model.get_weights()

    for round_num in range(num_rounds):
        local_weights_list = []
        print(f"Starting round {round_num + 1}/{num_rounds}...")

        # Local training on each client
        for client_idx, client_data in enumerate(client_datasets):
            # Clone the global model for local training
            local_model = create_model(global_model.input_shape[-1])
            local_model.set_weights(global_weights)

            # Compile the local model
            optimizer = tf.keras.optimizers.SGD(learning_rate)
            local_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

            # Train locally
            local_model.fit(client_data['X'], client_data['y'], epochs=local_epochs, verbose=0)

            # Collect local weights
            local_weights_list.append(local_model.get_weights())

        # Average local weights to update global model
        new_global_weights = [np.mean([local_weights[layer] for local_weights in local_weights_list], axis=0)
                              for layer in range(len(global_weights))]
        global_model.set_weights(new_global_weights)

        # Evaluate global model on test dataset
        global_loss, global_acc = global_model.evaluate(test_data['X'], test_data['y'], verbose=0)
        print(f"Global Model Accuracy: {global_acc:.4f} (after round {round_num + 1})")

    return global_model

# Initialize and compile the global model
input_dim = X_train.shape[1]
global_model = create_model(input_dim)
global_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

# Federated Averaging parameters
num_rounds = 5
local_epochs = 2
learning_rate = 0.01

# Train the global model using Federated Averaging
global_model = train_fedavg(global_model, client_data, test_data, num_rounds, local_epochs, learning_rate)
print("Federated training with FedAvg completed.")


0


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Starting round 1/5...
Global Model Accuracy: 0.9983 (after round 1)
Starting round 2/5...
Global Model Accuracy: 0.9983 (after round 2)
Starting round 3/5...
Global Model Accuracy: 0.9983 (after round 3)
Starting round 4/5...
Global Model Accuracy: 0.9983 (after round 4)
Starting round 5/5...
Global Model Accuracy: 0.9983 (after round 5)
Federated training with FedAvg completed.
